# GEOG0027 (2023/2024) Introduction to Google Earth Engine (GEE)


This practical will use Google Earth Engine (GEE)'s python library [EE](https://developers.google.com/earth-engine) and [geemap library](https://geemap.org/) to rapidly visualise, analyse and download LandSat Images. These two libraries are built to handle remote sensing (RS) data from the Cloud without physically downloading the data to our local computers (unless we really want to), and they also allow easy python coding, where only small modifications are needed before handling large datasets. GEE hosts many popular RS datasets on the Cloud, and details of its data catalog can be found at: https://developers.google.com/earth-engine/datasets.

GEE is natively written in JavaScript, and most web examples and the GEE official documentation are in this language. Here we use the python library to allow saveable code. The code is very similar between the two languages (function names are typically identical), with subtle differences between the syntax.

## Jupyter setup - only needs completing once
1) If you are using UCL's [Jupyter Hub](https://jupyter.data-science.rc.ucl.ac.uk/) (VPN required) to work with this chapter, please run the following command line in a `Terminal` (found top right of the Jupyter_Hub page under `New`) first:*

`conda config --add envs_dirs /shared/groups/jrole001/geog0027/envs/`
![terminal-add-env](images/terminal_add_env.png)

2) To download this notebook, to execute and modify the code, first create a directory, as explained in the previous session (ie `mkdir GEOG0027`), then navigate to it and then run the following command in a `Terminal`:

`wget https://github.com/UCL-EO/GEOG0027/raw/main/docs/Intro_to_GEE.ipynb`

3) Then, run `source activate geemap` in terminal to activate the geemap environment;

4) Run `jupyter nbextension install --py --symlink --user ipyleaflet` to install Leaflet extension, and then run `jupyter nbextension enable ipyleaflet --user --py` to enable the extension;

5) Close `Terminal` and double check if the `jupyter-leaflet extension` is enabled (especially if no visible output map from running the examples below):

![leaflet](images/leaflet.png)

6) Now open this notebook (Intro_to_GEE.ipynb) on jupyterhub. You will need to change the kernel under `Kernel -> Change Kernel -> Python [conda env:geog0027-geemap]`. The images on this notebook will not display, but that is fine. Use this page to read the notes, and modify and explore the code on the jupyter hub version. 

7) Run the follwing code cell on jupyter hub. If you cannot see a Google Map showing, then try to reboot Jupyter Hub to make the jupyter-leaflet extension work. Before that, make sure you have saved all your current working notebooks. Then reboot Jupyter by clicking on Control Panel - Stop My Server - Start My Server.

![rebootJupyter](images/reboot.png)

## First map
To create a map we first need to define a location. For this lesson aim to explore a location on the Earth of your choosing. Use google maps to pick a point and copy the coordinates.
![rebootJupyter](images/GEE_find_loc.png)

Enter the location in the cell below to define a map we use here. We start from displaying a basemap of the area. The first time when you use GEE, you will need to have a Google account and provide an authorization code. This can be a tricky task.

First you need to first to create a new Cloud Project. The easiest way to do this is to got to https://code.earthengine.google.com/, login in using a google account, and then select `Register a new Cloud Project` from the menu in the top right corner.

Then run cell below. Open the link provided in a new tab, and authorise a link to a google account. You can then select the Cloud Project registered previously.

Get the key code, paste it in box and press return.

In [1]:
import geemap, ee, os, numpy
import ipyleaflet

loc_coords = [51.49163903397572, -0.08313179193795397] ### EDIT THIS LINE WITH YOUR COORDS, [Lat,Lon] format
# loc_coords = [45.340728103851525, 23.23940244865895] ### EDIT THIS LINE WITH YOUR COORDS, [Lat,Lon] format


Map = geemap.Map(center=loc_coords, zoom=9)
Map

Map(center=[51.49163903397572, -0.08313179193795397], controls=(WidgetControl(options=['position', 'transparen…

A basic Google Map-like interface should be displayed here now. If you can't see anything, please ensure that the ipyleaflet nbextension has been enabled.

## Setting the Area Of Interest (AOI)
Now that we have a location to centre the Map on, we need to determine the size of the map. This can be done using a range in latitude/longitude using the function below. This cell just defines how a function works, and does not perform any actual calculations. To do this we then call the function.

In [3]:
def expand_coords(centre,lon_expand = 1.0,lat_expand = 0.5):
    lat1 = centre[0]-lat_expand
    lat2 = centre[0]+lat_expand
    lon1 = centre[1]-lon_expand
    lon2 = centre[1]+lon_expand
    return [lon1,lat1,lon2,lat2,]
    

This next piece of code will then calculate the map size. If the map isn't the correct size for your liking, add into the function alternative settings for 'lon_expand' and 'lat_expand' from those set in the previous cell.

In [4]:
rec_coords = expand_coords(loc_coords)
# rec_coords = expand_coords(loc_coords,lat_expand=0.7) ## example adjusted rectangle size
local_rec = ee.Geometry.Rectangle(rec_coords) 

## Examining the time series
Let's display a short 'movie' (a .gif file in fact) of how this area has changed over the past decades. This demonstrates the power of GEE - it is very quick to visualise a high volume of data.

In [6]:
Map_gif = geemap.Map(center=loc_coords, zoom=10)
Map_gif.add_landsat_ts_gif(roi=local_rec, start_year=1985, bands=['NIR', 'Red', 'Green'], frames_per_second=5)
Map_gif

Generating URL...
Please wait ...
The GIF image has been saved to: /Users/h/Downloads/landsat_ts_yjg.gif
Adding animated text to GIF ...
Adding GIF to the map ...
The timelapse has been added to the map.


Map(center=[45.340728103851525, 23.23940244865895], controls=(WidgetControl(options=['position', 'transparent_…

Next, we will compare such change by using a slider

In [5]:
landsat_ts = geemap.landsat_timeseries(roi=local_rec, start_year=1986, end_year=2020, \
                                       start_date='01-01', end_date='12-31')

layer_names = ['Landsat ' + str(year) for year in range(1986, 2021)]

geemap_landsat_vis = {
    'min': 0,
    'max': 3000,
    'gamma': [1, 1, 1],
    'bands': ['NIR', 'Red', 'Green']} # You can change the vis bands here

Map2 = geemap.Map()
Map2.ts_inspector(left_ts=landsat_ts, right_ts=landsat_ts, left_names=layer_names, right_names=layer_names, \
                 left_vis=geemap_landsat_vis, right_vis=geemap_landsat_vis)
Map2.centerObject(local_rec, zoom=10)
Map2

Map(center=[51.494062487824685, -0.08313179193795121], controls=(WidgetControl(options=['position', 'transpare…

We have previously defined a rectangle by coordinates, but we can also use existing shape/vector files to select Areas of Interest (not shown here but used for the coursework). 

## Load Landsat data collections from GEE
Now we can see the the buffered AOI displayed on the Map. Next, let's load some Landsat images for the selected area. I've defined here a python function called `display_landsat_collection` to do so. It automatically loads both the [surface reflectance](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_SR) and [annual NDVI](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_ANNUAL_NDVI) image collections from GEE's data catalog and also calculates the annual means for each band. 

You can skip most of the details of what's inside the code cell, but only to look at the first (and last) line of code. In order to run such function, you will need to choose a year (any year since 1984) and an AOI. In the following example, I choose year 2019 and a square box around London. 

In [6]:
landsat_vis_param = {
            'min': 0,
            'max': 3000,
            'bands': ['NIR', 'Red', 'Green']  # False Colour Composit bands to be visualised 
}
ndvi_colorized_vis = {
            'min': 0.0,
            'max': 1.0,
            'palette': [
            'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
            '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
            '012E01', '011D01', '011301']
}

In [7]:
def load_landsat_collection(inmap,year, aoi, cloud_tolerance = 3.0, 
                            DISPLAY_ON_MAP = False, MEDIAN_ONLY = False):
    '''This function allows GEE to display a Landsat data collection 
    from any year between 1984 and present year
    that fall within the AOI and cloud tolerance, e.g. 3.0%.
    There are two optional flag:
    When DISPLAY_ON_MAP is TRUE, display this layer onto Map;
    When return_series = 'MEDIAN_ONLY', only median SR is loaded into landsat_ts, and
    Setting this option to MEDIAN_ONLY would be faster than loading other collections. 
    '''
    assert year >= 1984
    
    def renameBandsETM(image):
        if year >=2013: #LS8
            bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7'] #, 'pixel_qa'
            new_bands = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'] #, 'pixel_qa'
        elif year <=1984:
            bands = ['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa']
            new_bands = ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2', 'pixel_qa']
        return image.select(bands).rename(new_bands)
        
    if not(MEDIAN_ONLY):
        if year >= 2013:
            layer_name = 'LC08' # LS8: 2013-now        
        elif year == 2012: # # LS7: 1999- , however SLC error >= 1999:
            layer_name = 'LE07' 
        elif year >=1984:
            layer_name = 'LT05' # LS5: 1984-2012
       
        collection_name_sr = f"LANDSAT/{layer_name}/C01/T1_SR" 
        # You can also use the following line, if interested in incorperating ndvi
        collection_name_ndvi = f"LANDSAT/{layer_name}/C01/T1_ANNUAL_NDVI" 

        all_sr_image = ee.ImageCollection(collection_name_sr) \
            .filterBounds(aoi) \
            .filterDate(f'{year}-01-01', f'{year}-12-31') \
            .filter(ee.Filter.lt('CLOUD_COVER', cloud_tolerance))\
            .sort('system:time_start') \
            .select('B[1-7]') \
            .sort('CLOUD_COVER')

        all_sr_image = all_sr_image.map(renameBandsETM) # rename bands with 'renameBandsETM' function
        
        # reduce all_sr_image to annual average per pixel
        mean_image = all_sr_image.mean()
        mean_image = mean_image.clip(aoi).unmask()

        ndvi_image = ee.ImageCollection(collection_name_ndvi)\
            .filterBounds(aoi) \
            .filterDate(f'{year}-01-01', f'{year}-12-31')\
            .select('NDVI')\
            .median()
        ndvi_image = ndvi_image.clip(aoi).unmask()

        #mean_image.addBands(ndvi_image, 'NDVI')
    
    # This line loads all annual median surface ref
    landsat_ts = geemap.landsat_timeseries(roi=aoi, start_year=year, end_year=year, \
                                       start_date='01-01', end_date='12-31')

    median_image = landsat_ts.first().clip(aoi).unmask()
    
    if DISPLAY_ON_MAP == True:
        
        if not(MEDIAN_ONLY):
            inmap.addLayer(ndvi_image, ndvi_colorized_vis, 'NDVI '+str(year),  opacity=0.9)
            inmap.addLayer(mean_image, landsat_vis_param, "Mean Ref "+str(year))
        inmap.addLayer(median_image, landsat_vis_param, "Median Ref "+str(year))

    if MEDIAN_ONLY:
        return median_image
    else:
        return all_sr_image, mean_image, median_image, ndvi_image 

Now the `load_landsat_collection` function has been defined, and we will run/execute it by calling the function name with appropriate input parameters (or 'arguments'). The output of such function will be returned to the variables on the LHS of the equal sign, i.e. all_image_2019, mean_2019, median_2019, and ndvi_2019 in this case.

You may get an error if you have chosen a location where no cloud free images can be found. In this case, increase the `cloud_tolerance`.

In [8]:
Map = geemap.Map(center=loc_coords, zoom=10)

# All you need to modify here is the YEAR below:
all_image_2019, mean_2019, median_2019, ndvi_2019 = load_landsat_collection(Map,2019,\
                                        local_rec, cloud_tolerance = 100,\
                                        DISPLAY_ON_MAP = True)
Map

Map(center=[51.49163903397572, -0.08313179193795397], controls=(WidgetControl(options=['position', 'transparen…

You should examine the functions under the `toolbar` and `layer` buttons on the top-right corner of the Map, e.g. use the `inspector` and `plotting` tools to check the data values, or use `layers` control to switch on/off layers or to adjust opacity.

We can also check the metadata from the Landsat image collection we just loaded from the Cloud. Have a look of the output. Any useful information?

In [11]:
first_image = all_image_2019.first() 

props = geemap.image_props(first_image)
print( props.getInfo())

{'CLOUD_COVER': 0, 'CLOUD_COVER_LAND': 0, 'EARTH_SUN_DISTANCE': 1.016734, 'ESPA_VERSION': '2_23_0_1b', 'GEOMETRIC_RMSE_MODEL': 7.162, 'GEOMETRIC_RMSE_MODEL_X': 4.84, 'GEOMETRIC_RMSE_MODEL_Y': 5.28, 'IMAGE_DATE': '2019-07-02', 'IMAGE_QUALITY_OLI': 9, 'IMAGE_QUALITY_TIRS': 9, 'LANDSAT_ID': 'LC08_L1TP_185029_20190702_20190706_01_T1', 'LEVEL1_PRODUCTION_DATE': 1562408365000, 'NOMINAL_SCALE': 30, 'PIXEL_QA_VERSION': 'generate_pixel_qa_1.6.0', 'SATELLITE': 'LANDSAT_8', 'SENSING_TIME': '2019-07-02T09:15:24.6234519Z', 'SOLAR_AZIMUTH_ANGLE': 136.611816, 'SOLAR_ZENITH_ANGLE': 26.959515, 'SR_APP_VERSION': 'LaSRC_1.3.0', 'WRS_PATH': 185, 'WRS_ROW': 29, 'system:asset_size': '593.230091 MB', 'system:band_names': ['Blue', 'Green', 'Red', 'NIR', 'SWIR1', 'SWIR2'], 'system:id': 'LANDSAT/LC08/C01/T1_SR/LC08_185029_20190702', 'system:index': 'LC08_185029_20190702', 'system:time_end': '2019-07-02 09:15:24', 'system:time_start': '2019-07-02 09:15:24', 'system:version': 1563395760808115}


The images shown so far are cropped agregates for the region we chose. To see the individual images involved in the composite, first get all the image names:

In [14]:
# gtest.getInfo()
image_list = all_image_2019.aggregate_array('system:id')
Im_list = image_list.getInfo()
# Im_list.sort()
Im_list

['LANDSAT/LC08/C01/T1_SR/LC08_201025_20190224',
 'LANDSAT/LC08/C01/T1_SR/LC08_201025_20190920',
 'LANDSAT/LC08/C01/T1_SR/LC08_202024_20190420',
 'LANDSAT/LC08/C01/T1_SR/LC08_202025_20190420',
 'LANDSAT/LC08/C01/T1_SR/LC08_202025_20190215',
 'LANDSAT/LC08/C01/T1_SR/LC08_203024_20190918',
 'LANDSAT/LC08/C01/T1_SR/LC08_201024_20190224',
 'LANDSAT/LC08/C01/T1_SR/LC08_202024_20190215',
 'LANDSAT/LC08/C01/T1_SR/LC08_201025_20190515',
 'LANDSAT/LC08/C01/T1_SR/LC08_200024_20190305',
 'LANDSAT/LC08/C01/T1_SR/LC08_201024_20190515',
 'LANDSAT/LC08/C01/T1_SR/LC08_200024_20190524',
 'LANDSAT/LC08/C01/T1_SR/LC08_203024_20190513',
 'LANDSAT/LC08/C01/T1_SR/LC08_202024_20190826',
 'LANDSAT/LC08/C01/T1_SR/LC08_201025_20191225',
 'LANDSAT/LC08/C01/T1_SR/LC08_200025_20190217',
 'LANDSAT/LC08/C01/T1_SR/LC08_200025_20190422',
 'LANDSAT/LC08/C01/T1_SR/LC08_202024_20190130',
 'LANDSAT/LC08/C01/T1_SR/LC08_200024_20191202',
 'LANDSAT/LC08/C01/T1_SR/LC08_202024_20190725',
 'LANDSAT/LC08/C01/T1_SR/LC08_201024_201

In [10]:
# gtest.getInfo()
image_list = ndvi_2019.aggregate_array('system:id')
image_list.getInfo()

AttributeError: 'Image' object has no attribute 'aggregate_array'

Is this list longer or shorter than you expected? 
> What happens when you adust `cloud_tolerance = ?` in the previous function call?
>
> Copy one of the images from the list generated by the previous code cell into the cell below. 

We will start by getting the band names. 

In [14]:
### NOTE, this code will not work until you edit it !!!
image_name = 'LANDSAT/LC08/C01/T1_SR/LC08_186028_20190810'
single_image = ee.Image(image_name)

single_image.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B10',
 'B11',
 'sr_aerosol',
 'pixel_qa',
 'radsat_qa']

If we want to find this image on the original USGS website (see in the final section) then we can use the original satellite retrieval Time, Path and Row to find it. These are printed below.

In [15]:

print('Date/Time = {}'.format(single_image.get('SENSING_TIME').getInfo()))
print('Path = {}'.format(single_image.get('WRS_PATH').getInfo()))
print('Row = {}'.format(single_image.get('WRS_ROW').getInfo()))


Date/Time = 2019-08-10T09:21:23.5874220Z
Path = 186
Row = 28


Previously the bands were renamed, but for a single image, we need to use the original band names. These are typically B[1-8], though you may need to adjust these depending on the year chosen.

First we will look at a single band of a single image.

In [16]:
#### adjust 'band_use' to look at a different band.
band_use = 'B7'
landsat_single_vis = {'min': 0,'max': 3000,'bands': [band_use]}

Map.addLayer(single_image,landsat_single_vis,image_name+band_use)

Map

Map(bottom=376524.0, center=[45.23017314147431, 22.65243530273438], controls=(WidgetControl(options=['position…

Now we combine several bands to create a Red/Green/Blue image.
> Can you change this to a False Colour image? 
>
>Can you add all the images that make up the previous mean/median composites?

In [17]:
landsat_vis = {'min': 0,'max': 3000,'bands': ['B4', 'B3', 'B2']}

Map.addLayer(single_image,landsat_vis,image_name)

Map

Map(bottom=11912.0, center=[46.15700496290803, 22.840576171875004], controls=(WidgetControl(options=['position…

Next, examine the mean, median surface reflectance and/or NDVI layers we've visualized.
>Which one is most useful? 
>
>How do these relate to the individual images?

One particular issue that can arise is taking the 'mean' of individiual NDVI images. Instead we can take the 'NDVI' of mean images (or medians).


In addition to switching layers on and off, adjusting opacity, we can also use python code to some simple mathmatical operations, e.g. calculating the differences:

In [18]:
mean_ndvi = mean_2019.normalizedDifference(['NIR', 'Red'])
median_ndvi = median_2019.normalizedDifference(['NIR', 'Red'])
median_ndwi = median_2019.normalizedDifference(['Green','NIR'])

Map.addLayer(mean_ndvi, ndvi_colorized_vis, 'NDVI from Mean LS')
Map.addLayer(median_ndvi, ndvi_colorized_vis, 'NDVI from Median LS')
Map.addLayer(mean_ndvi.subtract(median_ndvi), {'min': -0.2,'max': 0.2}, 'Diff in NDVI')
Map.addLayer(median_ndwi, ndvi_colorized_vis, 'NDWI from Median LS')
Map

Map(bottom=23572.0, center=[46.069419674968536, 21.928710937500004], controls=(WidgetControl(options=['positio…

## Downloading Images
What we've preformed so far, involves working entirely in the GEE cloud. This is great for quick image exploration and combining images for certain areas. Sometimes however we want to download the images for prosperity, or for finer analysis beyond the GEE function library.

The `ee_export_image()` function allows us to download all the images we've explored so far.

In [11]:
### this will download the single_iamge shown above
### remember to change the filename for multiple downloads
geemap.ee_export_image(single_image,filename='single_image.tif')

Generating URL ...
Please wait ...
Data downloaded to /Users/h/GEOG/GEOG0027/GEOG0027/docs/single_image.tif


In [14]:
### this will download the NDVI composite. 
### you may need to adjust the scale parameter to allow the download
geemap.ee_export_image(median_2019, filename='Median_NDVI_2019.tif', \
                            scale=200, region=local_rec) 

Generating URL ...
Please wait ...
Data downloaded to /Users/h/GEOG/GEOG0027/GEOG0027/docs/Median_NDVI_2019.tif


How do you rate the quality of the downloaded images? Do you think Google is restricting bandwidth and not sending the full image? Well there is a way to get the original images.

# Download raw images (USGS)

In this section, you will learn how to order Landsat data from the USGS, including how to search only for the area we are interested in, or how to go to the exact image you want. 

>Can you get the original versions of the single images found using GEE?

## Downloading and visualisation tools 

There are several tools available to you for browsing and ordering Landsat data. See the [USGS](https://www.usgs.gov/land-resources/nli/landsat/landsat-data-access?qt-science_support_page_related_con=0#qt-science_support_page_related_con) for a full description of the data products and where to order them.

You will most often be interested in **surface reflectance** products. These can be ordered through the [USGS Earth Resources Observation and Science (EROS) Center Science Processing Architecture (ESPA) On Demand Interface](https://espa.cr.usgs.gov/).


The first time you visit the ordering service, you will need to [register for an account](https://ers.cr.usgs.gov/register/). Make sure you remember the usename and password you used for registration!

Once at the [USGS Earth Resources Observation and Science (EROS) Center Science Processing Architecture (ESPA) On Demand Interface](https://espa.cr.usgs.gov/) site, make yourself aware of the information on [surface reflectance](https://www.usgs.gov/land-resources/nli/landsat/landsat-surface-reflectance?qt-science_support_page_related_con=0#qt-science_support_page_related_con).



## Select Landsat Scenes

To select a particular landsat scene, go to the [Earth Explorer]() site.

Enter the place you are interested in, in the `Search Criteria` (e.g. try: London, England) or select an area on the graphic using the `Use Map` polygon option. To get the image we found using GEE, this needs to be consistent with previous options.


https://earthexplorer.usgs.gov
Under `Data Sets`, look in `Landsat -> Collection 1 Level 1` and check all of the boxes that have surface reflectance data. This will be for different sensors in the Landsat series (LS4-5, LS-7, LS-8).

You *may* wish to set a cloud cover limitation (e.g. 10%) under `Search Criteria -> Cloud Cover`.
You *may* wish to set a time period (e.g. covering the previously made composite) under `Search Criteria -> Date Range`.


Click on 'Results' and look at a few of the images. In particular, you should look at [which `PATH` and `ROW`](https://landsat.usgs.gov/landsat_acq) is appropriate for the area you want data for. This could, for example be

        Path 201
        Row 24
        
Can you use the Path and Row information found previously using GEE? This is set under `Additional Criteria`.
        
        
Now, you can go back to the `Search Criteria` and enter just the path/row that you want, and/or just go through and add the files you want to download to the basket. This is the bar you have:
Footprint, Overlay, Compare, Metadata, Download, Bulk Download, Order Scene, Remove
 
![options](images/USGS_options.png)

The first two are good for looking at, checking the cloud cover, and image cover. Choose Download to get the full image. Select the GeoTIFF, the largest file. Once this image is donwloaded, it can be opened using ENVI.


This ends the work for the first practical on GEE. Well done.

Now go back and adjust the options for each of the previous tasks. Can you explore a different location, or time?

## Summary

In this practical we have introduced Google Earth Engine (GEE) and how to obtain satellite images from the LandSat Archive.

In particular we have:
* Introduced using jupyter notebooks to run code examples and view a GEE map.
* Introduced the GEE code library and used it to rapidly view a time series of images from an arbitrary location from the Earths surface.
* Explored the individual images that make up a yearly composite.
* Explored the individual bands that make up a LandSat image.

From the information obtained from exploring GEE we can find out
* An individual images retrieval time and date.
* Each images specific retrieval details (Path and Row).
* Used these details to obtain the original image from USGS.